In [1]:
from mesa.batchrunner import BatchRunner, BatchRunnerMP
from mesa.time import RandomActivation
from model import *
from customizeTime import *
import pickle
import numpy as np
import uuid

from customizeTime import *
import env_settings

TYPE_MAP = {0:'vert',
            1:'horz'}

settings.MUTATION_PRO = 0.01

class A_2(GenAgent):
    def muta_genetic_info(self):
        # no need for mutation
        pass
    
        
class M_2(GenModel):
    def get_agent_config(self):
        return A_2
    
    def init_env(self):
        #no influence
        return [self.env_val] * 5000
    
    def get_schedule(self):
        return RandomActivationMutationWithConstrain
    #
    def __init__(self, N, width, height, specific_kargs):
        #mut_var, init_var, env_val
        self.model_name = 'M2'
        
        init_ratio = specific_kargs['init_ratio']
        self.env_val = specific_kargs['env_val']
        
        
        settings.INIT_MEAN = specific_kargs['gene_mean']
        settings.INIT_SD = specific_kargs['gene_sd']
        settings.INIT_LOW = settings.INIT_MEAN - 1 * 2 * settings.INIT_SD
        settings.INIT_HIGH = settings.INIT_MEAN + 1 * 2 * settings.INIT_SD

        settings.POPU_MUTA_SD = specific_kargs['muta_sd']
        settings.POPU_MUTA_LOW = -2 * settings.POPU_MUTA_SD
        settings.POPU_MUTA_HIGH = 2 * settings.POPU_MUTA_SD
        
        self.specific_kargs_str = '_'.join([str(i) for i in (self.env_val, settings.INIT_SD ,settings.POPU_MUTA_SD)])
        super().__init__(N, width, height, init_ratio)
        

    
    def model_level_record_init(self):
        data_file_name = check_path("./res/")
        data_file_name = check_path(data_file_name + self.model_name)
        data_file_name = check_path(data_file_name + "/" + self.specific_kargs_str)
        data_file_name = data_file_name + '/' + str(uuid.uuid1())
        print('data in file ', data_file_name)
        self.data_file = open(data_file_name, 'w')
        self.data_file.write('#' + '\t'.join(
            ('step',
             TYPE_MAP[0], TYPE_MAP[0] + '_beta', TYPE_MAP[0] + '_gene', 'var',  
             TYPE_MAP[1], TYPE_MAP[1] + '_beta', TYPE_MAP[1] + '_gene', 'var'))+ '\n')
       
    def get_popu_beta(self, gen_type):
        return np.mean([agent.self_gen_cof for agent in self.schedule.agents if agent.gen_type == gen_type])
    
    def get_gene_mean(self, gen_type):
        return [np.mean(agent.gen_info) for agent in self.schedule.agents if agent.gen_type == gen_type]
    
    def get_popu_gene_mean(self, gene_mean):
        return np.mean(gene_mean)

    def get_popu_gene_var(self, gene_mean):
        return np.var(gene_mean)

    def collect_data(self):
        ver_mean = self.get_gene_mean(0)
        hor_mean = self.get_gene_mean(1)
        return (self.schedule.steps,
                self.get_ver_popu_size(), self.get_popu_beta(0), self.get_popu_gene_mean(ver_mean), self.get_popu_gene_var(ver_mean),
                self.get_her_popu_size(), self.get_popu_beta(1), self.get_popu_gene_mean(hor_mean), self.get_popu_gene_var(hor_mean))
        
        
    def step(self):
        data = self.collect_data()
        #print(data)
        self.data_file.write('\t'.join([str(d) for d in data]) + '\n')
        if self.schedule.steps % 100 == 0: 
            self.data_file.flush()
        super().step()
        
        


In [ ]:
import sys

#0.01, 0.005, 0.001
sys.argv = 'm2 1 0.01'.split(' ')

env_val, muta_sd = float(sys.argv[1]), float(sys.argv[2])

fixed_params = {'N':2000, 'width':100, 'height':100}

variable_params = {'specific_kargs': [{'env_val': env_val , 'init_ratio':0.5, 'gene_mean':0.5, 'gene_sd':muta_sd, 'muta_sd':muta_sd}]}

batch_run = BatchRunnerMP(
    M_2, 2,
    variable_parameters=variable_params,
    fixed_parameters=fixed_params,
    iterations=2,
    max_steps=2000,
)
batch_run.run_all()

0it [00:00, ?it/s]

data in file  ./res/M2/1.0_0.01_0.01/a951cb6e-cecf-11e9-9322-f2189850a77c
data in file  ./res/M2/1.0_0.01_0.01/a953137a-cecf-11e9-b6bb-f2189850a77c
0 population size:  2000
0 population size:  2000
1 population size:  2613
1 population size:  2666
2 population size:  3384
2 population size:  3285
3 population size:  4043
3 population size:  4082
4 population size:  4698
4 population size:  4752
5 population size:  5237
5 population size:  5305
6 population size:  5727
6 population size:  5713
7 population size:  6089
7 population size:  6063
8 population size:  6367
8 population size:  6258
9 population size:  6478
9 population size:  6392
10 population size:  6590
10 population size:  6506
11 population size:  6688
11 population size:  6627
12 population size:  6644
12 population size:  6670
13 population size:  6718
13 population size:  6693
14 population size:  6676
14 population size:  6734
15 population size:  6627
15 population size:  6805
16 population size:  6670
16 population 

In [4]:
np.clip(np.random.normal(loc=settings.GENE_INIT_MEAN, scale=settings.GENE_INIT_SD, size=10), settings.GENE_INIT_LOW, settings.GENE_INIT_HIGH)

array([-0.4, -0.4, -0.4, -0.4, -0.4,  0.4, -0.4, -0.4, -0.4, -0.4])